# TUGAS PRAKTIKUM 8 METODE NUMERIK
__Solusi Akar Persamaan Tak-Linear__

```Nama: Antonius Aditya Rizky Wijaya```

```NIM: G5402221003```

<hr style="border:2px solid black"> </hr>

### Function metode yang harus dijalankan terlebih dahulu :

In [89]:
# Fungsi untuk mencari interval akar dari suatu fungsi yang diduga memiliki lebih dari satu akar
function interval_akar(f, a, b, subinterval)
    interval_akar = []
    nilai_a = a
    while nilai_a < b
        nilai_b = nilai_a + subinterval
        if f(nilai_a) * f(nilai_b) < 0
            push!(interval_akar, (nilai_a, nilai_b))
        end
        nilai_a += subinterval
    end
    return interval_akar
end;

### 1. Metode _Bisection_
Metode ini memiliki syarat, yaitu nilai evaluasi fungsi pada ujung selang $ [a,b] $ yaitu $ f(a) $ dan $ f(b) $ memiliki **tanda yang berbeda**.

In [90]:
function bisection(f,a,b)
    # Definisikan nilai toleransi dan maksimum iterasi yang telah ditentukan.
    delta = 10^-7;
    maxi = 100;
    flag = 1;
    M = Array{Float64}(undef, 0, 5);
    # Cek Syarat kekonvergenan, yaitu f(a) dan f(b) harus berbeda tanda.
    fa = f(a);
    fb = f(b);
    if fa*fb>0
        c = "error: f(a) dan f(b) harus berbeda tanda";
        flag = 2; 
        return;
    end
    k = 1
    # Mulai proses metode bisection
    while k<=maxi
        # Isi dengan rumus bisection untuk mencari nilai c.
        c  = (b+a)/2; 
        fc = f(c); 
        M = [M; [k-1 a c b fc] ];
        # Analisa nilai tengah yang memiliki nilai tanda yang berbeda dengan titik ujung selang.
        if fc == 0
            a = c;
            b = c;
        elseif fa*fc>0
            a = c;
            fa = fc;
        else
            b = c;
            fb = fc;
        end
        # Kriteria pemberhentian
        if b-a < delta || abs(fc) < delta
            flag = 0;
            break;
        end
        k+=1
    end
    return c,flag, M
end;

### 2. Metode _Regula Falsi_
Sama seperti metode _bisection_, metode ini memiliki syarat yaitu nilai evaluasi fungsi pada ujung selang $ [a,b] $ yaitu $ f(a) $ dan $ f(b) $ memiliki **tanda yang berbeda**.

In [91]:
#=
METODE REGULA FALSI UNTUK MENCARI AKAR PERSAMAAN
 [c,flag,M] = regulaFalsi(f,a,b)
 Input  : f    -> fungsi f
          a,b  -> titik ujung selang [a,b]
 Output : c    -> solusi numerik dari nilai hampiran akar
          flag -> 0 -> toleransi terpehuhi
                  1 -> maksimum iterasi terpenuhi
                  2 -> error: f(a) dan f(b) memiliki tanda yang sama 
          M    -> Matriks yang berisi nilai iterasi, a, c, b, dan f(c)
=#
function regulaFalsi(f,a,b)
    # Definisikan nilai toleransi dan maksimum iterasi yang ditentukan.
    delta = 10^-7;
    maxi = 100;
    flag = 1;
    M = Array{Float64}(undef, 0, 5);
    # Cek Syarat kekonvergenan, yaitu f(a) dan f(b) harus berbeda tanda.
    fa = f(a);
    fb = f(b);
    if fa*fb > 0
        c = "error : fa fb harus beda tanda";
        flag = 2; 
        return;
    end 
    # Mulai proses metode regula falsi
    for k = 1:maxi
        c = b-fb*(b-a)/(fb-fa);# Isi dengan rumus nilai c regula falsi.
        fc = f(c); 
        dx = min(c-a,b-c);
        M = [M ; [k-1 a c b fc] ]; 
        # Analisa nilai tengah dengan titik ujung selang.
        if fc == 0
            a = c;
            b = c;
        elseif fa*fc>0
            a = c;
            fa= fc;
        else
            b = c;
            fb= fc;
        end
        # Kriteria pemberhentian
        if abs(fc) < delta || abs(dx)< delta
            flag = 0; break;
        end
    end
    return c,flag,M
end;

### 3. Metode _Newton-Raphson_
$p_k = p_{k-1} - \frac{f(p_{k-1})}{f'(p_{k-1})}$

In [92]:
#= %%METODE NEWTON-RAPHSON 
% [pk, flag, M] = newtonRaphson (f,df,p0)
% Input  : f    -> fungsi f
%        : df   -> fungsi turunan dari f
%          p0   -> starting value
% Output : pk   -> nilai akar
%          flag -> tanda : 0 -> berhasil
%                          1 -> gagal 
%          M    -> matriks yang berisi nilai iterasi, hampiran akar dan
%                  galat
=#
function newtonRaphson(f,df,p0)
    # Definisikan nilai toleransi, maksimum iterasi dan tebakan awal yang telah ditentukan.
    delta = 10^-7;
    maxi = 100;
    pk = p0
    M = [0 pk NaN];
    # Mulai langkah iterasi
    flag = 1;
    for k = 2:maxi
        # Rumus metode Newton-Raphson
        pk1 = pk
        pk = pk1 - f(pk1)/df(pk1); 
        # Hitung nilai galat mutlak dan relatif
        err = abs(pk-pk1);
        rel = 2*err/(abs(pk)+eps());
        M = [M; [k-1 pk err]]
        # Kriteria penghentian iterasi jika galat memenuhi toleransi.
        if err<delta || rel<delta
            flag = 0;
            break
        end
    end 
    return pk, flag, M
end;

### 4. Metode _Secant_
$ p_{k+1}=p_k - f(p_{k}) \dfrac{p_k-p_{k-1}}{f(p_k)-f(p_{k-1})}$

In [93]:
#=
%%METODE SECANT
% [pk, flag, M] = secant(f,p0,p1)
% Input  : f    -> fungsi f 
%          p0,p1-> starting value
% Output : pk   -> nilai akar
%          flag -> tanda 
%                  0 -> berhasil
%                  1 -> gagal 
%          M    -> matriks yang berisi nilai iterasi, hampiran akar dan galat
=#
function secant(f,p0,p1)
    # Definisikan nilai toleransi, maksimum iterasi dan tebakan awal yang telah ditentukan.
    delta = 10^-12;
    maxi = 100; 
    M = [0 p0 NaN
         1 p1 NaN];
    p = [p0 p1];
    pk = p1;
    flag = 1; 
    # Mulai langkah iterasi
    for k = 2:maxi
        # rumus metode secant 
        pk=p[k]-f(p[k])*(p[k]-p[k-1])/(f(p[k])-f(p[k-1]));
        p = [p pk]
        # Hitung nilai galat mutlak dan relatif
        err=abs(p[k+1]-p[k]);
        rel=2*err/(abs(p[k]) + eps());
        M = [M; [k pk err]] 
        # Kriteria penghentian iterasi jika galat memenuhi toleransi.
        if err<delta || rel<delta
            flag = 0;
            break
        end
    end
    return pk, flag, M
end;

<hr style="border:2px solid black"> </hr>

### __Soal 1__
Berapa banyak solusi dari persamaan berikut? $$sin(10x)−x = 0$$
Carilah semua solusi akar persamaan tak linear di atas menggunakan metode _bracketing_!

In [94]:
f(x) = sin(10x)-x;
a = -1.0; b = 1.0;
f(a), f(b)

(1.5440211108893698, -1.5440211108893698)

Berdasarkan nilai $f(a)$ dan $f(b)$, maka dipastikan terdapat akar persamaan $f(x) = 0$ pada interval $[-1 , 1]$

In [95]:
function f(x)
    return sin(10x)-x
end;

a = -1.0;
b = 1.0;
subinterval = 0.1
interval = interval_akar(f, a, b, subinterval)
println("Interval yang memiliki tepat satu akar:")
for i in interval
    println("> ", i)
end

Interval yang memiliki tepat satu akar:
> (-0.9, -0.8)
> (-0.8, -0.7000000000000001)
> (-0.30000000000000016, -0.20000000000000015)
> (-1.3877787807814457e-16, 0.09999999999999987)
> (0.19999999999999987, 0.2999999999999999)
> (0.6999999999999998, 0.7999999999999998)
> (0.7999999999999998, 0.8999999999999998)


**Terkonfirmasi** bahwa persamaan $sin(10x)−x = 0$ memiliki 7 nilai akar yang masing - masing terdapat pada interval ```[-0.9 , -0.8]``` , ```[-0.8 , -0.7]``` , ```[-0.3 , -0.2]``` , ```[0.0 , 0.1]``` , ```[0.2 , 0.3]``` , ```[0.7 , 0.8]``` , dan ```[0.8 , 0.9]```

### __> Dengan metode Bisection__

In [96]:
c1, flag1, M1 = bisection(f, -0.9, -0.8);
c2, flag2, M2 = bisection(f, -0.8, -0.7);
c3, flag3, M3 = bisection(f, -0.3, -0.2);
c4, flag4, M4 = bisection(f, 0.0 , 0.1);
c5, flag5, M5 = bisection(f, 0.2, 0.3);
c6, flag6, M6 = bisection(f, 0.7, 0.8);
c7, flag7, M7 = bisection(f, 0.8, 0.9);

println("\e[1mSolusi fungsi sin(10x) - x = 0 dengan metode bisection:\e[0m\n")
println("Solusi 1 = ", c1)
println("Solusi 2 = ", c2)
println("Solusi 3 = ", c3)
println("Solusi 4 = ", c4)
println("Solusi 5 = ", c5)
println("Solusi 6 = ", c6)
println("Solusi 7 = ", c7)

Solusi fungsi sin(10x) - x = 0 dengan metode bisection:

Solusi 1 = -0.8423203468322755
Solusi 2 = -0.7068174362182618
Solusi 3 = -0.28523416519165046
Solusi 4 = 9.5367431640625e-8
Solusi 5 = 0.28523416519165046
Solusi 6 = 0.7068174362182618
Solusi 7 = 0.8423203468322755


### __> Dengan metode Regula Falsi__

In [97]:
c1, flag1, M1 = regulaFalsi(f, -0.9, -0.8);
c2, flag2, M2 = regulaFalsi(f, -0.8, -0.7);
c3, flag3, M3 = regulaFalsi(f, -0.3, -0.2);
c4, flag4, M4 = regulaFalsi(f, 0.0 , 0.1);
c5, flag5, M5 = regulaFalsi(f, 0.2, 0.3);
c6, flag6, M6 = regulaFalsi(f, 0.7, 0.8);
c7, flag7, M7 = regulaFalsi(f, 0.8, 0.9);

println("\e[1mSolusi fungsi sin(10x) - x = 0 dengan metode regula falsi:\e[0m\n")
println("Solusi 1 = ", c1)
println("Solusi 2 = ", c2)
println("Solusi 3 = ", c3)
println("Solusi 4 = ", c4)
println("Solusi 5 = ", c5)
println("Solusi 6 = ", c6)
println("Solusi 7 = ", c7)

Solusi fungsi sin(10x) - x = 0 dengan metode regula falsi:

Solusi 1 = -0.8423203805355834
Solusi 2 = -0.7068174367544002
Solusi 3 = -0.2852341891820905
Solusi 4 = 0.0
Solusi 5 = 0.2852341891820905
Solusi 6 = 0.7068174367544002
Solusi 7 = 0.8423203805355834


<hr style="border:2px solid black"> </hr>

### __Soal 2__
Implementasikan metode _bisection_, Newton, dan _secant_ untuk menyelesaikan persamaan $$e^{-x} = x$$

setidaknya untuk salah satu akar persamaannya.

__(a)__ Kriteria pemberhentian apa yang tepat digunakan masing-masing metode?

__(b)__ Apa laju kekonvergenan yang diperoleh dari setiap kasus?

__(c)__ Bandingkan hasil yang diperoleh! (dari segi solusi dan laju kekonvergenan)

In [98]:
f(x) = exp(-x) - x;
df(x) = -exp(-x) - 1;
a = 0.0; b = 1.0;
f(a), f(b)

(1.0, -0.6321205588285577)

Berdasarkan nilai $f(a)$ dan $f(b)$, maka dipastikan terdapat akar persamaan $f(x) = 0$ pada interval $[0 , 1]$

In [99]:
function f(x)
    return exp(-x)-x
end;

a = 0.0;
b = 1.0;
subinterval = 0.1
interval = interval_akar(f, a, b, subinterval)
println("Interval yang memiliki tepat satu akar:")
for i in interval
    println("> ", i)
end

Interval yang memiliki tepat satu akar:
> (0.5, 0.6)


Dengan demikian, untuk metode _bisection_ saya akan gunakan interval $[0.5 , 0.6]$ sebagai interval solusinya. Kemudian saya akan gunakan $p_0 = 0.5$ dan $p_1 = 0.6$ sebagai tebakan awal untuk metode _newton-raphson_ dan _secant_.


In [100]:
p0 = 0.5; p1 = 0.6;

### __> Dengan metode Bisection__

In [101]:
c, flag_bisection, M_bisection = bisection(f, p0, p1)
println("\e[1mMetode Bisection:\e[0m")
if flag_bisection == 0
    println("\e[1mSolusi: \e[0m", c)
    M_bisection
else
    println("Metode bisection tidak konvergen")
end

Metode Bisection:
Solusi: 0.5671432495117188


16×5 Matrix{Float64}:
  0.0  0.5       0.55      0.6        0.0269498
  1.0  0.55      0.575     0.6       -0.0122951
  2.0  0.55      0.5625    0.575      0.00728282
  3.0  0.5625    0.56875   0.575     -0.00251721
  4.0  0.5625    0.565625  0.56875    0.00238003
  5.0  0.565625  0.567187  0.56875   -6.92822e-5
  6.0  0.565625  0.566406  0.567187   0.0011552
  7.0  0.566406  0.566797  0.567187   0.000542917
  8.0  0.566797  0.566992  0.567187   0.000236806
  9.0  0.566992  0.56709   0.567187   8.37594e-5
 10.0  0.56709   0.567139  0.567187   7.23791e-6
 11.0  0.567139  0.567163  0.567187  -3.10223e-5
 12.0  0.567139  0.567151  0.567163  -1.18922e-5
 13.0  0.567139  0.567145  0.567151  -2.32718e-6
 14.0  0.567139  0.567142  0.567145   2.45536e-6
 15.0  0.567142  0.567143  0.567145   6.40931e-8

In [104]:
#Hitung nilai galat
Ek_bisection = abs.(c .- M_bisection[:,3])

16-element Vector{Float64}:
 0.01714324951171875
 0.007856750488281161
 0.004643249511718794
 0.0016067504882811834
 0.00151824951171875
 4.425048828116118e-5
 0.0007369995117187944
 0.0003463745117188166
 0.00015106201171888323
 5.340576171886102e-5
 4.577636718794409e-6
 1.9836425781183387e-5
 7.62939453125e-6
 1.5258789062277955e-6
 1.5258789063388178e-6
 0.0

Hitung nilai rasio galat $ |E_{k+1}|/|E_k|^R $. Dengan $R=\frac{1}{2}$.

In [105]:
R = 0.5;
RasioGalat = abs.(Ek_bisection[2:end]) ./ abs.(Ek_bisection[1:end-1].^R)

15-element Vector{Float64}:
 0.0600062363352685
 0.05238422717370047
 0.02357965408586542
 0.0378764205386507
 0.0011356551958098674
 0.11079193985386855
 0.012758873351237675
 0.00811675244843416
 0.004345207112454477
 0.0006263935357376703
 0.009271345374360298
 0.0017130039439619176
 0.0005524271727939513
 0.001235264711084163
 0.0

Berdasarkan hasil yang didapatkan, nilai $ |E_{k+1}|/|E_k|^R $ konvergen menuju suatu bilangan, yaitu $0.0012$. 

Sehingga, metode _Bisection_ memiliki laju kekonvergenan $R=\frac{1}{2}$ untuk _single root_.

### __> Dengan metode Newton-Raphson__

In [106]:
pk_newton, flag_newton, M_newton = newtonRaphson(f, df, p0)
println("\e[1mMetode Newton-Raphson:\e[0m")
if flag_newton == 0
    println("\e[1mSolusi: \e[0m", pk_newton)
    M_newton
else
    println("Metode Newton-Raphson tidak konvergen")
end

Metode Newton-Raphson:
Solusi: 0.567143290409784


5×3 Matrix{Float64}:
 0.0  0.5       NaN
 1.0  0.566311    0.066311
 2.0  0.567143    0.000832162
 3.0  0.567143    1.25375e-7
 4.0  0.567143    2.88658e-15

In [107]:
# Hitung nilai galat
Ek_newton = abs.(pk_newton .-M_newton[:,2])

5-element Vector{Float64}:
 0.06714329040978395
 0.0008322872125657854
 1.2537492177777665e-7
 2.886579864025407e-15
 0.0

Hitung nilai rasio galat $ |E_{k+1}|/|E_k|^R $. Dengan $R=2$ untuk _single root_ dan $R=1$ untuk _double root_. 

Berdasarkan hasil komputasi sebelumnya, persamaan $e^{-x} = x$ memiliki 1 akar penyelesaian (_single root_), sehingga pilih $R=2$.

In [108]:
R = 2;
RasioGalat = abs.(Ek_newton[2:end]) ./ abs.(Ek_newton[1:end-1].^R)

4-element Vector{Float64}:
 0.18461542369661624
 0.18099402198565437
 0.18363786190783848
 0.0

Berdasarkan hasil yang didapatkan, nilai $ |E_{k+1}|/|E_k|^2 $ konvergen menuju suatu bilangan, yaitu $0.18$. 

Sehingga metode _Newton-Raphson_ memiliki laju kekonvergenan $R=2$ untuk _single root_.

### __> Dengan metode Secant__

In [109]:
pk_secant, flag_secant, M_secant = secant(f, p0, p1)
println("\e[1mMetode Secant:\e[0m")
if flag_secant == 0
    println("\e[1mSolusi: \e[0m", pk_secant)
    M_secant
else
    println("Metode Secant tidak konvergen")
end

Metode Secant:
Solusi: 0.5671432904097838


7×3 Matrix{Float64}:
 0.0  0.5       NaN
 1.0  0.6       NaN
 2.0  0.567545    0.0324554
 3.0  0.567141    0.000403668
 4.0  0.567143    2.37391e-6
 5.0  0.567143    1.72355e-10
 6.0  0.567143    1.11022e-16

In [110]:
# Hitung nilai galat
Ek_secant = abs.(pk_secant .-M_secant[:,2])

7-element Vector{Float64}:
 0.06714329040978384
 0.03285670959021614
 0.0004012944275175734
 2.3737362090381353e-6
 1.7235479710109303e-10
 1.1102230246251565e-16
 0.0

Hitung nilai rasio galat $ |E_{k+1}|/|E_k|^R $. Pilih $ R=\dfrac{1+\sqrt{5}}{2} $

In [111]:
R = (1+sqrt(5))/2;
RasioGalat = abs.(Ek_secant[2:end]) ./ abs.(Ek_secant[1:end-1].^R)

6-element Vector{Float64}:
 2.5976085083783413
 0.1008360449499241
 0.7432664945397057
 0.2173525981802943
 0.6969619358315214
 0.0

Berdasarkan hasil yang didapatkan, nilai $ |E_{k+1}|/|E_k|^R $ konvergen menuju suatu bilangan, yaitu $0.7$. 

Sehingga, metode _Secant_ memiliki laju kekonvergenan $ R=\dfrac{1+\sqrt{5}}{2}\approx1.618 $ untuk _single root_.

### Kesimpulan dan jawaban nomor 2a, 2b, dan 2c:

Solusi dari persamaan $e^{-x} = x$ :
- Metode Bisection: $0.5671432495117188$, dengan $15$ kali iterasi dan akurasi $7$ angka dibelakang koma.
- Metode Newton-Raphson: $0.567143290409784$, dengan $4$ kali iterasi dan akurasi $7$ angka dibelakang koma.
- Metode Secant: $0.5671432904097838$, dengan $6$ kali iterasi dan akurasi $12$ angka dibelakang koma.

__(a) Kriteria pemberhentian apa yang tepat digunakan masing-masing metode?__

Berdasarkan materi yang dipaparkan pada modul dan notebook praktikum:
- Kriteria pemberhentian metode _bisection_ adalah ketika panjang interval $|b-a|$ lebih kecil dari toleransi $delta$, atau ketika nilai fungsi di titik tengah $f(c)$ mendekati nol.

- Kriteria pemberhentian metode _newton-raphson_ adalah ketika perubahan relatif antara iterasi ke $k$ dan $k+1$ lebih kecil dari toleransi $delta$, atau ketika nilai fungsi mendekati nol.

- Kriteria pemberhentian metode _secant_ adalah ketika perubahan relatif antara iterasi ke $k$ dan $k+1$ lebih kecil dari toleransi $delta$, atau ketika nilai fungsi mendekati nol.

__(b) Apa laju kekonvergenan yang diperoleh dari setiap kasus?__

Berdasarkan hasil komputasi diatas:
- Laju kekonvergenan metode _bisection_ untuk persamaan $e^{-x} = x$ adalah $R=\frac{1}{2}$.

- Laju kekonvergenan metode _newton-raphson_ umumnya memiliki nilai $R=2$ untuk _single root_ dan $R=1$ untuk _double root_. Namun karena persamaan $e^{-x} = x$ memiliki 1 akar penyelesaian maka laju kekonvergenannya  adalah $R=2$.

- Laju kekonvergenan metode _secant_ untuk persamaan $e^{-x} = x$ adalah $R=\dfrac{1+\sqrt{5}}{2}\approx1.618$.

__(c) Bandingkan hasil yang diperoleh! (dari segi solusi dan laju kekonvergenan)__

- Metode _bisection_ menjamin konvergensi terhadap solusinya, namun dengan iterasi yang lebih banyak dan laju kekonvergenan yang lebih lambat dibandingkan dengan _newton-raphson_ dan _secant_. Selain itu, metode _bisection_ lebih stabil dan tidak memerlukan turunan fungsi.

- Metode _newton-raphson_ cenderung memberikan solusi lebih cepat dengan iterasi yang lebih sedikit dibandingkan dengan metode _bisection_ dan _secant_. Namun, hasilnya dapat bervariasi tergantung pada tebakan awal yang diberikan dan memerlukan turunan fungsinya.

- Metode _secant_ memiliki konvergensi yang lebih cepat daripada metode _bisection_ tetapi lebih lambat daripada _newton-raphson_. Metode ini tidak memerlukan perhitungan turunan seperti _newton-raphson_, sehingga cocok untuk menyelesaikan fungsi yang turunannya sulit dicari, tetapi hasilnya lebih bergantung pada pemilihan tebakan awal yang tepat.

<hr style="border:2px solid black"> </hr>

### __Soal 3__
A certain nonlinear spring develops a restoring force given by $$F = c_1x+c_3x^3+c_5x^5$$

where F is in Kg force and x is in cm. The values of the constants are
$$c_1 = 5.25 kg/cm,$$

$$c_3 = 0.60 kg/cm,$$

$$c_5 = 0.0118 kg/cm.$$

If a 12.5 kg weight is placed on the spring, how far will it compress?
(where standard gravity is 9.80665 $m/s^2$)

In [112]:
c1 = 5.25
c3 = 0.60
c5 = 0.0118
massa = 12.5
gravitasi = 9.80665
f(x) = c1*x + c3*x^3 + c5*x^5 - massa * gravitasi
df(x) = c1 + 3*c3*x^2 + 5*c5*x^4

a = 1.0
b = 5.0
f(a), f(b)

(-116.721325, 15.541875000000005)

Berdasarkan nilai $f(a)$ dan $f(b)$, maka dipastikan terdapat akar persamaan $f(x) = 0$ pada interval $[1 , 5]$

In [113]:
function f(x)
    return c1*x + c3*x^3 + c5*x^5 - massa * gravitasi
end;

a = 1.0;
b = 5.0;
subinterval = 0.1
interval = interval_akar(f, a, b, subinterval)
println("Interval yang memiliki tepat satu akar:")
for i in interval
    println("> ", i)
end

Interval yang memiliki tepat satu akar:
> (4.8, 4.8999999999999995)


Dengan demikian, untuk metode _bisection_ saya akan gunakan interval $[4.8 , 4.9]$ sebagai interval solusinya. Kemudian saya akan gunakan $p_0 = 4.8$ dan $p_1 = 4.9$ sebagai tebakan awal untuk metode _newton-raphson_ dan _secant_.


In [114]:
p0 = 4.8; p1 = 4.9;

### __> Dengan metode Bisection__

In [118]:
jarak_kompresi, flag, M = bisection(f, p0, p1)
if flag == 0
    println("\e[1mJarak kompresi pegas saat diberikan massa sebesar 12.5 kg\e[0m: ", jarak_kompresi, " cm")
    M
else
    println("\e[1mMetode bisection tidak konvergen\e[0m")
end

Jarak kompresi pegas saat diberikan massa sebesar 12.5 kg: 4.8122727394104 cm


20×5 Matrix{Float64}:
  0.0  4.8      4.85     4.9       2.99567
  1.0  4.8      4.825    4.85      1.00359
  2.0  4.8      4.8125   4.825     0.0178594
  3.0  4.8      4.80625  4.8125   -0.472448
  4.0  4.80625  4.80938  4.8125   -0.227507
  5.0  4.80938  4.81094  4.8125   -0.104877
  6.0  4.81094  4.81172  4.8125   -0.0435221
  7.0  4.81172  4.81211  4.8125   -0.0128347
  8.0  4.81211  4.8123   4.8125    0.00251154
  9.0  4.81211  4.81221  4.8123   -0.00516178
 10.0  4.81221  4.81226  4.8123   -0.00132517
 11.0  4.81226  4.81228  4.8123    0.000593173
 12.0  4.81226  4.81227  4.81228  -0.000366002
 13.0  4.81227  4.81227  4.81228   0.000113585
 14.0  4.81227  4.81227  4.81227  -0.000126209
 15.0  4.81227  4.81227  4.81227  -6.3118e-6
 16.0  4.81227  4.81227  4.81227   5.36366e-5
 17.0  4.81227  4.81227  4.81227   2.36624e-5
 18.0  4.81227  4.81227  4.81227   8.6753e-6
 19.0  4.81227  4.81227  4.81227   1.18175e-6

### __> Dengan metode Newton-Raphson__

In [119]:
jarak_kompresi, flag, M = newtonRaphson(f, df, p0)
if flag == 0
    println("\e[1mJarak kompresi pegas saat diberikan massa sebesar 12.5 kg\e[0m: ", jarak_kompresi, " cm")
    M
else
    println("\e[1mMetode Newton-Raphson tidak konvergen\e[0m")
end

Jarak kompresi pegas saat diberikan massa sebesar 12.5 kg: 4.812272724370745 cm


4×3 Matrix{Float64}:
 0.0  4.8      NaN
 1.0  4.81231    0.0123147
 2.0  4.81227    4.19394e-5
 3.0  4.81227    4.88276e-10

### __> Dengan metode Secant__

In [120]:
jarak_kompresi, flag, M = secant(f, p0, p1)
if flag == 0
    println("\e[1mJarak kompresi pegas saat diberikan massa sebesar 12.5 kg\e[0m: ", jarak_kompresi, " cm")
    M
else
    println("\e[1mMetode secant tidak konvergen\e[0m")
end

Jarak kompresi pegas saat diberikan massa sebesar 12.5 kg: 4.812272724370745 cm


7×3 Matrix{Float64}:
 0.0  4.8      NaN
 1.0  4.9      NaN
 2.0  4.81198    0.0880233
 3.0  4.81227    0.000288929
 4.0  4.81227    7.13209e-6
 5.0  4.81227    5.86128e-10
 6.0  4.81227    8.88178e-16

### Kesimpulan nomor 3:

Dari ketiga metode yang digunakan, didapatkan jarak kompresi pegas saat diberikan massa sebesar $12.5$ kg adalah sekitar $4.8122727$ cm. Namun jika dilihat dari jumlah iterasi yang diperlukan untuk mendapat solusi tersebut, metode _newton-raphson_ lebih efisien karena hanya membutuhkan $3$ kali iterasi saja.

<hr style="border:2px solid black"> </hr>